# Erklärung Preprocesse

Dieser Abschnitt erklärt die meisten Methoden des Objektes Preprocesse. Weggelassen wurden Methoden, welche nur für die Entwicklung benötigt wurden und im Produktivcode nicht aufgerufen werden (Debugging Methoden).
### Disclaimer
Der Code ist nicht lauffähig, dafür existieren die Python Skripte. Das Notebook stellt lediglich eine Dokumentation dar.

### Verbesserungen, welche an Main vorgenommen wurden
- Nichts besonderes

### Import aller spezifischen Bibliotheken und Module, Klassendefinition und init Methode
Hier wird der Gauss Kernel als festes Attribut der Klasse gesetzt.

In [ ]:
import cv2 as cv
import numpy as np

class Preprocess():

    GAUSS_KERNEL = (5, 5)
    
    def __init__(self) -> None:
        pass

### Gauss, Canny Edge und Threshold
Diese Funktionalität wurde in eigene prepocess Methoden ausgelagert. Die jeweiligen Variablen (canny_lower, canny_upper, threshold) sind in der Config Datei ausgelagter, um sie möglichst leicht anpassen zu können. Nur der Gauss Kernel wurde hier fest definiert.

In [ ]:
def gauss(self, img):
    return cv.GaussianBlur(img, self.GAUSS_KERNEL, 0)

def canny(self, img, canny_lower, canny_upper):
    return cv.Canny(img, canny_lower, canny_upper)

def threshold(self, img, thresh):
    return cv.threshold(img, thresh[0], thresh[1], cv.THRESH_BINARY)[1]

### segmenation
In dieser Funktion wird aus dem zu verarbeitenden Bild die Region of Interest ausgeschnitten. Das bedeutet, nur dieser Bildbereich bleibt enthalten. Dies hat den Sinn, bspw. die obere Hälfte des Bildes abzuschneiden. Hier befinden sich keine Fahrspuren.

In [ ]:
def segmentation(self, img, roi, negative = False):
    # Define a blank matrix that matches the image height/width.
    mask = np.zeros_like(img)

    match_mask_color = 255
    
    # Fill inside the polygon
    vertices = self._generateCoordinatesRectangle(img, roi)
    cv.fillPoly(mask, np.array([vertices], np.int32), match_mask_color)
    if negative: mask = cv.bitwise_not(mask)
    
    # Returning the image only where mask pixels match
    masked_image = cv.bitwise_and(img, mask)
    
    return masked_image

### generate coordinates rectangle
Hier werden die Koordinaten für die segmentation erstellt. Die eigentlichen Werte kommen aus der Konfigurationsdatei, hier werden sie in das richtige Format gebracht.

In [ ]:
def _generateCoordinatesRectangle(self, img, roi):
    # Generate the region of interest
    dim = img.shape
    height = dim[0]
    width = dim[1]
    roi = [
        (roi[0][0], height + roi[0][1]),
        ((width / 2) + roi[1][0], (height / 2) + roi[1][1]),
        ((width / 2) + roi[2][0], (height / 2) + roi[2][1]),
        (width + roi[3][0], height + roi[3][1]),
    ]

    return roi

### map color
Hier wird das Gelb der Fahrspur auf einen vollen weiß-wert übertragen. Dadurch kann in späteren Verarbeitungsschritten (Threshold) die Fahrspurerkennung deutlich verbessert werden.
Die Grenzwerte für den Gelb-ton sind Video-spezifisch und darum in der Konfigurationsdatei ausgelagert.

In [ ]:
def map_color(self, img, lower, upper):
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    # Threshold of the color in HSV space
    lower = np.array(lower)
    upper = np.array(upper)

    # preparing the mask to overlay
    mask = cv.inRange(hsv, lower, upper)

    # The black region in the mask has the value of 0,
    # so when multiplied with original image removes all non-blue regions
    result = cv.bitwise_and(img, img, mask = mask)
    result = self.threshold(result, (1, 255))
    result = cv.bitwise_or(img, result)
    return result